In [1]:
#import libraries.
import numpy as np
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense
from tensorflow.python.keras.utils.data_utils import Sequence
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
import keras
from keras import regularizers
import sklearn
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
import sounddevice as sd
import tensorflow as tf
from mir_eval import separation 
from pystoi.stoi import stoi 
import h5py
from keras.callbacks import LearningRateScheduler
from keras.layers.normalization import BatchNormalization
from keras.layers import Activation
import math
from sklearn.utils import shuffle
from sklearn import preprocessing
import soundfile as sf

Using TensorFlow backend.


In [2]:
#import libraries.
# import matplotlib.pyplot as plt
# from tabulate import tabulate
import time
import os
import librosa
from librosa.core import stft, istft
import time
import pickle
from keras import backend as K
from tensorflow.keras.models import load_model
# from keras.models import load_model

In [3]:
def reconstruct(wave,angle):
    recon1 = wave*np.cos(angle)+wave*np.sin(angle)*1j
#     recon = np.sqrt(np.power(10, wave))
#     recon1 = recon*np.cos(angle)+recon*np.sin(angle)*1j
    recon = librosa.core.istft((recon1.T), hop_length=512, win_length=2048, window='hann')
    return recon

# Loading Model

In [4]:
estimator_A = load_model('Models/Two_stage/July8/saved-model-adam-046-0.0357.hdf5')

Instructions for updating:
`normal` is a deprecated alias for `truncated_normal`
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.cast instead.


In [5]:
data_series = str(0)

In [6]:
h5f = h5py.File('TSP/Organized/concatenated/Test_TSP_mixed_FFT'  + data_series + '.hdf5','r')
data_shape = h5f['Test_TSP_mixed_FFT' + data_series].shape
print(data_shape)
h5f.close()

(1025, 21675)


# Data 

In [7]:
# all
test_point_start0 = 0
test_point_stop0 = data_shape[1]

In [8]:
# FM
test_point_start0 =  0
test_point_stop0 = int(data_shape[1]/3) - 155

In [46]:
# FF
test_point_start0 =  int(data_shape[1]/3) - 155
test_point_stop0 = int(2 * data_shape[1]/3) - 110

In [65]:
# MM
test_point_start0 =  int(2 * data_shape[1]/3) - 110
test_point_stop0 = data_shape[1]

In [12]:
# small proportion
test_point_start0 = 19000
test_point_stop0 = 20500

In [9]:
h5f = h5py.File('TSP/Organized/concatenated/Test_TSP_mixed_FFT'  + data_series + '.hdf5','r')
test_input = h5f['Test_TSP_mixed_FFT' + data_series][:, test_point_start0 : test_point_stop0]
print(test_input.shape)
h5f.close()

(1025, 7070)


In [10]:
h5f = h5py.File('TSP/Organized/concatenated/Test_TSP_clean_FFTs'  + data_series + '.hdf5','r')
test_target = h5f['Test_TSP_clean_FFTs' + data_series][:, test_point_start0 : test_point_stop0]
print(test_target.shape)
h5f.close()

(2050, 7070)


In [11]:
h5f = h5py.File('TSP/Organized/concatenated/Test_TSP_mixed_phase'  + data_series + '.hdf5','r')
mixed_phase = h5f['Test_TSP_mixed_phase' + data_series][:, test_point_start0 : test_point_stop0]
print(mixed_phase.shape)
h5f.close()

(1025, 7070)


In [12]:
h5f = h5py.File('TSP/Organized/concatenated/Test_TSP_clean_phases'  + data_series + '.hdf5','r')
clean_phase = h5f['Test_TSP_clean_phases' + data_series][:, test_point_start0 : test_point_stop0]
print(clean_phase.shape)
h5f.close()

(2050, 7070)


In [13]:
norms = np.zeros(test_input.shape[1])
for i in range(test_input.shape[1]):
    norms[i] = np.linalg.norm(test_input[:,i])

In [14]:
test_input = preprocessing.normalize(test_input, norm='l2', axis=0, copy=True)

In [15]:
prediction = estimator_A.predict(test_input.T)

In [16]:
test_input = np.multiply(test_input,norms)

In [17]:
prediction0 = np.multiply(test_input.T, prediction[:,0:1025])
prediction1 = np.multiply(test_input.T, prediction[:,1025:])

In [18]:
fft_samples = 1025

In [19]:
speaker1 = reconstruct(prediction0, mixed_phase.T)
speaker2 = reconstruct(prediction1, mixed_phase.T)

In [20]:
clean_phase.shape

(2050, 7070)

In [21]:
clean_recon1 = reconstruct(test_target[0:fft_samples,:].T, clean_phase[0:fft_samples,:].T)
clean_recon2 = reconstruct(test_target[fft_samples:,:].T, clean_phase[fft_samples:,:].T)

In [22]:
mixed_recon = reconstruct(test_input.T, mixed_phase.T)

In [24]:
st_recon1 = stoi(speaker1, clean_recon1, 48000, extended=False)
st_recon2 = stoi(speaker2, clean_recon2, 48000, extended=False)

In [25]:
st_mixed1 = stoi(mixed_recon, clean_recon1, 48000, extended=False)
st_mixed2 = stoi(mixed_recon, clean_recon2, 48000, extended=False)

In [25]:
print('STOI_recon1=', st_recon1)
print('STOI_recon2=', st_recon2)
print('STOI_mixed1=', st_mixed1)
print('STOI_mixed2=', st_mixed2)

STOI_recon1= 0.7484800260927625
STOI_recon2= 0.6617332535895908
STOI_mixed1= 0.6283168544837595
STOI_mixed2= 0.5435481313152923


In [33]:
sdr_recon1, sir_recon1, sar_recon1, perm_recon1 = separation.bss_eval_sources(mixed_recon, clean_recon1, compute_permutation=True)

In [34]:
sar_recon1

array([5.24252842])

In [26]:
sdr_recon1, sir_recon1, sar_recon1, perm_recon1 = separation.bss_eval_sources(speaker1, clean_recon1, compute_permutation=True)
sdr_recon2, sir_recon2, sar_recon2, perm_recon2 = separation.bss_eval_sources(speaker2, clean_recon2, compute_permutation=True)

In [27]:
sdr_mixed1, sir_mixed1, sar_mixed1, perm_mixed1 = separation.bss_eval_sources(mixed_recon, clean_recon1, compute_permutation=True)
sdr_mixed2, sir_mixed2, sar_mixed2, perm_mixed2 = separation.bss_eval_sources(mixed_recon, clean_recon2, compute_permutation=True)

In [28]:
print('SDR_recon1=', sdr_recon1)
print('SDR_recon2=', sdr_recon2)
print('SDR_mixed1=', sdr_mixed1)
print('SDR_mixed2=', sdr_mixed2)

SDR_recon1= [5.61239878]
SDR_recon2= [4.72275549]
SDR_mixed1= [1.36286961]
SDR_mixed2= [-0.16238854]


In [249]:
print('SIR_recon1=', sir_recon1)
print('SIR_recon2=', sir_recon2)
print('SIR_mixed1=', sir_mixed1)
print('SIR_mixed2=', sir_mixed2)

SIR_recon1= [inf]
SIR_recon2= [inf]
SIR_mixed1= [inf]
SIR_mixed2= [inf]


In [56]:
sd.play(speaker1, 48000)

In [57]:
sd.play(speaker2, 48000)

In [227]:
sd.play(clean_recon1, 48000)

In [226]:
sd.play(clean_recon2, 48000)

In [169]:
sd.play(mixed_recon, 48000)

In [80]:
#  optional to hear better
speaker1 = speaker1/max(speaker1)
speaker2 = speaker2/max(speaker1)
clean_recon1 = clean_recon1/max(clean_recon1)
clean_recon2 = clean_recon2/max(clean_recon2)
mixed_recon = mixed_recon/max(mixed_recon)

In [45]:
sf.write('TSP/Predicted/Voices/FM/speaker1_FM.wav', speaker1, 48000)
sf.write('TSP/Predicted/Voices/FM/speaker2_FM.wav', speaker2, 48000)
sf.write('TSP/Predicted/Voices/FM/clean1_FM.wav', clean_recon1, 48000)
sf.write('TSP/Predicted/Voices/FM/clean2_FM.wav', clean_recon2, 48000)
sf.write('TSP/Predicted/Voices/FM/mixed_FM.wav', mixed_recon, 48000)

In [64]:
sf.write('TSP/Predicted/Voices/FF/speaker1_FF.wav', speaker1, 48000)
sf.write('TSP/Predicted/Voices/FF/speaker2_FF.wav', speaker2, 48000)
sf.write('TSP/Predicted/Voices/FF/clean1_FF.wav', clean_recon1, 48000)
sf.write('TSP/Predicted/Voices/FF/clean2_FF.wav', clean_recon2, 48000)
sf.write('TSP/Predicted/Voices/FF/mixed_FF.wav', mixed_recon, 48000)

In [81]:
sf.write('TSP/Predicted/Voices/MM/speaker1_MM.wav', speaker1, 48000)
sf.write('TSP/Predicted/Voices/MM/speaker2_MM.wav', speaker2, 48000)
sf.write('TSP/Predicted/Voices/MM/clean1_MM.wav', clean_recon1, 48000)
sf.write('TSP/Predicted/Voices/MM/clean2_MM.wav', clean_recon2, 48000)
sf.write('TSP/Predicted/Voices/MM/mixed_MM.wav', mixed_recon, 48000)

In [30]:
from pypesq import pesq
score = pesq(clean_recon1, speaker1, 48000)
print(score)

ValueError: sample rate must be 16000 or 8000

In [98]:
def loss_DNNC(y_true = test_target.T, y_pred = test_target.T, Lambda = 0.05):
    loss = K.sum(K.square(y_true - y_pred)) 
    - Lambda * K.sum(K.square(y_true[0:257,:] - y_pred[0:257,:]) + K.square(y_true[257:,:] - y_pred[257:,:]))
    return loss

In [99]:
estimator_C = load_model('Models/Two_stage/Model_C0.h5', 
                          custom_objects={'loss_DNNC': loss_DNNC})

In [100]:
sec_set = np.concatenate((speaker1 , speaker2), axis=1)

In [101]:
sec_set = preprocessing.normalize(sec_set, norm='l2', axis=1, copy=True)
test_target = preprocessing.normalize(test_target, norm='l2', axis=1, copy=True)

In [102]:
prediction2 = estimator_C.predict(sec_set)

In [103]:
prediction2.shape

(18634, 514)

In [104]:
s1_C = reconstruct(prediction2[:,0:257], angle_mixed)
s2_C = reconstruct(prediction2[:,257:], angle_mixed)

In [105]:
clean_recon1 = reconstruct(test_target[:,0:fft_samples], angle_clean[:,0:fft_samples])
clean_recon2 = reconstruct(test_target[:,fft_samples:], angle_clean[:,fft_samples:])

In [106]:
st_recon1 = stoi(s1_C, clean_recon1, 16000, extended=False)
st_recon2 = stoi(s2_C, clean_recon2, 16000, extended=False)

In [107]:
sdr_recon1, sir_recon1, sar_recon1, perm_recon1 = separation.bss_eval_sources(s1_C, clean_recon1, compute_permutation=True)
sdr_recon2, sir_recon2, sar_recon2, perm_recon2 = separation.bss_eval_sources(s2_C, clean_recon2, compute_permutation=True)

In [108]:
print('SDR_recon1=', sdr_recon1)
print('SDR_recon2=', sdr_recon2)
print('SDR_mixed=', sdr_mixed)

SDR_recon1= [-4.51841408]
SDR_recon2= [-2.20552222]
SDR_mixed= [-4.78920035]


In [109]:
print('STOI_recon1=', st_recon1)
print('STOI_recon2=', st_recon2)
print('STOI_mixed=', st_mixed)

STOI_recon1= 0.2992286684160478
STOI_recon2= 0.36715637272317214
STOI_mixed= 0.3243004405902902


In [50]:
sd.play(s1_C[start:stop]*2000, 16000)